<a href="https://colab.research.google.com/github/harjeet88/llm-course/blob/main/4_module/1_simple_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install the Google Generative AI SDK
!pip install -q openai google-generativeai faiss-cpu sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.7 MB/s eta 0:00:00


In [2]:
# Step 2: Import libraries
import os
import openai
from openai import OpenAI
from google.colab import userdata
#import google.generativeai as genai
from IPython.display import display, Markdown

In [3]:
# Step 3
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
GEMINI_API_KEY=GOOGLE_API_KEY
gemini_model='gemini-2.5-flash'
gemini_api_endpoint="https://generativelanguage.googleapis.com/v1beta/openai/"

In [4]:
# Step 4: Initialize the model (using Gemini flash)
# Configure OpenAI client to use Google's Gemini endpoint
gemini = OpenAI(api_key=GOOGLE_API_KEY, base_url=gemini_api_endpoint)

In [6]:
# Set Hugging Face API token (get a free token from huggingface.co)
#os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACE_API_KEY')
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [7]:
!echo "i am harjeet. i work at bla bla company. i am from Bangalore India." > "sample_ai_textbook.txt"

In [8]:
# Step 1: Load and chunk document
with open("sample_ai_textbook.txt", "r") as file:
    text = file.read()

In [9]:
# Simple chunking by splitting into paragraphs (approx. 500 chars)
chunks = [text[i:i+500] for i in range(0, len(text), 500)]
print(f"Created {len(chunks)} chunks")

Created 1 chunks


In [10]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
#from huggingface_hub import InferenceClient

In [11]:
# Step 2: Create embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(chunks, convert_to_numpy=True)
dimension = embeddings.shape[1]  # Embedding size (e.g., 384)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
# Step 3: Store embeddings in FAISS
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
print("Embeddings stored in FAISS")

Embeddings stored in FAISS


In [13]:
def retrive_chunk(question):
  question_embedding = model.encode([question])[0]
  D, I = index.search(np.array([question_embedding]), k=1)  # Find top 1 chunk
  retrieved_chunk = chunks[I[0][0]]
  print("Retrieved chunk:", retrieved_chunk[:100], "...")
  return retrieved_chunk

In [14]:
def getAnswer(question) :
  retrieved_chunk = retrive_chunk(question)
  system="you are a LLM using RAG to answer user questions. you are impersonating the person mentioned in RAG. so strictly build answers based on context. If you donot know answer, say i dont know  "
  prompt = f"Context: {retrieved_chunk}\n\nQuestion: {question}\nAnswer:"
  answer = gemini.chat.completions.create(
      model=gemini_model,
      messages=[
          {"role": "system", "content": system},
          {"role": "user", "content": prompt}
      ]
  ).choices[0].message.content
  return answer

In [15]:
# Step 4: Query the system
question = "where do you work?"
answer=getAnswer(question)
print("Question:", question)
print("Answer:", answer)

Retrieved chunk: i am harjeet. i work at bla bla company. i am from Bangalore India.
 ...
Question: where do you work?
Answer: I work at bla bla company.
